# Loading and splitting Dataset
The goal of this notebook is to continue the development of the DeepTropism model using a Pytorch.<br>
The dataset was already created on the previous notebook and all the HIV-1 env V3 loop sequences where aligned.


In [1]:
# Libraries used on the analysis
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import random

First we load the Dataframe with all the sequences published on referenced articles.<br>
The D


In [8]:
df = pd.read_csv('/home/gabriel/Documents/Repos/DeepTropism/datasets/dataset_profile_final.tsv', 
                 sep='\t', names=['seq_name', 'dataset', 'label', 'sequence_aligned'])
df.head()

,seq_name,dataset,label,sequence_aligned
0,B.FR.83.HXB2_LAI_IIIB_BRU.K03455,LANL,-,CTRPNNN-TRKRI-RIQRGPGRAFVTI-----GK-IGNMRQAHC
1,A1.CD.02.LA01AlPr.KU168256,LANL,-,CIRPNNN-TRKGI-GI--GPGQTFYAA-----DAIIGNIRHAYC
2,A1.CM.08.886_24.KP718928,LANL,-,CSRPNNN-TRRSI-RI--GPGQSFYAT-----GEIIGDIREARC
3,A1.ES.15.100_117.KY496622,LANL,-,CTRPGNN-TRTSI-RI--GPGQAFYAT-----GDIIGDIRKAYC
4,A1.KE.11.DEMA111KE002.KF716474,LANL,-,CTRPNNN-TRKSV-RI--GPGQAFFAT-----GEVIGKIRKAYC


In [9]:
df.shape

(9748, 4)

In [6]:
### Check the size of the column of the alig

In [10]:
set(df['sequence_aligned'].apply(len))

{44}

### Show the original datasets present on Dataframe

In [13]:
df.dataset.value_counts()

newdb         2998
cm            2679
hivcopred     2335
geno2pheno    1188
webpssm        350
LANL           198
Name: dataset, dtype: int64

In [14]:
df = df[df.label != 'validation']
df.shape

(9748, 4)

In [15]:
df.label.value_counts()

CCR5     7705
CXCR4     937
R5X4      908
-         198
Name: label, dtype: int64

In [16]:
df.label_numeric.value_counts()

AttributeError: 'DataFrame' object has no attribute 'label_numeric'

# Filtering the dataset by unique sequences
To improve the quality of our trainning and avoid bias we are going to create a dataset with only unique sequences from the original Dataframe.
This is going to be the dataset for the development of our model based on Deep Neural Network.

In [7]:
df_unique = df.drop_duplicates(subset=['sequence_aligned'], keep='first')

In [65]:
df_unique.head()

,seq_name,dataset,label,sequence,sequence_aligned,label_numeric
0,RAB014775,newdb,CCR5,CTRPSNNTRTGITIGPGQVWYRTGDIIGDIRKAYC,CTRPSNNT-RTGI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
1,RAB014776,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGDIRQAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
2,RAB014778,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGDIRKAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0
3,RAB014781,newdb,CCR5,CTRPSNNTRTSVTIGPGQVWYRTGDIIGDIRQAYC,CTRPSNNT-RTSV---T--I--G-P--G--Q-VWY---R-T--GDI...,0
4,RAB014834,newdb,CCR5,CTRPSNNTRTSITIGPGQVWYRTGDIIGNIRKAYC,CTRPSNNT-RTSI---T--I--G-P--G--Q-VWY---R-T--GDI...,0


In [9]:
df_unique.label_numeric.value_counts()

0    2783
1     825
Name: label_numeric, dtype: int64

In [10]:
df_unique.label.value_counts()

CCR5     2783
R5X4      485
CXCR4     340
Name: label, dtype: int64

In [8]:
df_unique.shape

(3608, 6)

## Creating Dataframes for comparing performance against other methods
In order to evaluate our model against the others already published, we are going to create separate Dataframes for each method filterinf by the 'dataset' column

In [5]:
df.dataset.value_counts()

newdb         2998
cm            2679
hivcopred     2335
geno2pheno    1188
webpssm        350
Name: dataset, dtype: int64

In [6]:
df_newdb = df[df.dataset == 'newdb']
df_cm = df[df.dataset == 'cm']
df_hivcopred = df[df.dataset == 'hivcopred']
df_geno2pheno = df[df.dataset == 'geno2pheno']
df_webpssm = df[df.dataset == 'webpssm']

# Spliting the Dataset for Cross Validation
We are going to create indices and set it to variables to make our cross validation reproducible. Our dataset is going to consist on:<br>
* Trainning = 80 %
* Validation = 10 %
* Test = 10 %

In [104]:
# Create a list of indices and shuffle it using seed
random.seed(42)
size = df_unique.shape[0]
list_indices = list(range(size))
random.shuffle(list_indices)

In [105]:
# Now create the list of indices for trainning, validation and test
test_indices = list_indices[:int(size/10)]
train_val_indices = list_indices[int(size/10):]

In [106]:
len(test_indices)

360

In [107]:
len(train_val_indices)

3248

In [108]:
assert len(train_val_indices) + len(test_indices) == len(list_indices), "Splitting indices with error"

## Creating the Dataloaders for trainning

In [11]:
def get_array_from_sequence(protein_sequence):
    """
    Function to convert a protein sequence into a tensor.
    Each amino acid is represented by an numpy array of zeros of size 26,
    and the dict_aa_pos defines the position to be converted to 1.
    
    The function iterates over the protein sequences and stacks the arrays.
    At the end the arrays are linearized and converted to a tensor of size
    n x 26, with n the size of the protein.
    
    If the character is not present on the dict_aa_pos (eg. '-') the respective
    array is formed by zeros, and represents a missing value.
    """
    dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}
    
    f_array = np.zeros(26)
    for aa in protein_sequence:
        arr = np.zeros(26)
        if dict_aa_pos.get(aa):
            arr[dict_aa_pos.get(aa)] = 1
        f_array = np.vstack((f_array, arr))
    f_array = np.delete(f_array, 0,0)
    
    #return torch.from_numpy((f_array.flatten()).astype(float))
    return f_array.flatten().astype(float)

In [13]:
dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}

In [14]:
len(dict_aa_pos.keys())

26

In [114]:
# Create list to append data from the df
list_data = []
list_labels = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df.iterrows():
    list_data.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels.append(int(row.label_numeric))

In [111]:
len(list_data) == len(list_labels)

True

In [115]:
# For Test set
test_data = []
test_label = []
for j in test_indices:
    test_data.append(list_data[j])
    test_label.append(np.array(list_labels[j]))

test_tensor_x = torch.stack([torch.from_numpy(i) for i in test_data]) # transform to torch tensors
test_tensor_y = torch.stack([torch.from_numpy(i) for i in test_label])

test_dataset = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y) # create your test dataset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64) # create your dataloader


In [116]:
# Now we define the
len(train_val_indices)

3248

In [117]:
crossval_val_indices = np.array_split(train_val_indices,5)

In [118]:
len(crossval_val_indices[0])

650

In [119]:
# For Trainning and validation set
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader

Cross Validation: 1
Cross Validation: 2
Cross Validation: 3
Cross Validation: 4
Cross Validation: 5


## Define the Deep Neural Network Architecture


In [128]:
class DeepTropism_1(nn.Module):
    def __init__(self):
        super(DeepTropism_1, self).__init__()
        self.linear1 = nn.Linear(1560,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,2)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
model = DeepTropism_1().float()
#model = model.float()
model

DeepTropism_1(
  (linear1): Linear(in_features=1560, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=2, bias=True)
)

### For showing the metrics of the model

In [156]:
def show_metrics(y_true, y_score):
    # True positive
    tp = np.sum(y_true * y_score)
    # False positive
    fp = np.sum((y_true == 0) * y_score)
    # True negative
    tn = np.sum((y_true==0) * (y_score==0))
    # False negative
    fn = np.sum(y_true * (y_score==0))

    # True positive rate (sensitivity or recall)
    tpr = tp / (tp + fn)
    # False positive rate (fall-out)
    fpr = fp / (fp + tn)
    # Precision
    precision = tp / (tp + fp)
    # True negatvie tate (specificity)
    tnr = 1 - fpr
    # F1 score
    f1 = 2*tp / (2*tp + fp + fn)
    # ROC-AUC for binary classification
    auc = (tpr+tnr) / 2
    # MCC
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    print("True positive: ", tp)
    print("False positive: ", fp)
    print("True negative: ", tn)
    print("False negative: ", fn)

    print("True positive rate (recall): ", tpr)
    print("False positive rate: ", fpr)
    print("Precision: ", precision)
    print("True negative rate (Specificity): ", tnr)
    print("F1: ", f1)
    print("ROC-AUC: ", auc)
    print("MCC: ", mcc)

In [129]:
# Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [130]:
# For Trainning and validation set DeepTropism_1
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader
    
    # Instantiante new model
    #model = DeepTropism_1().float()
        
    # Define Cross Validation Trainning Loop
    for epoch in range(400):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainning_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            #print(running_loss)
            #if i % 3239 == 0:    # print every 3239 mini-batches
            if epoch % 20 == 0 and i % 3248 == 0:
                print('[%d, %3d] loss: %.9f' %
                      (epoch + 1, i + 1, running_loss / 50))
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in validation_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy for validation set {n + 1}: {round(100.0 * correct/total, 2)}%')
                
                # Evaluate model against test set
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in test_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')

                
            running_loss = 0.0
            
            
    torch.save(model.state_dict(), f'model_cv{n+1}.ptb')
    print('Finished Training')
    

Cross Validation: 1
[1,   1] loss: 0.014718180
Neural Network accuracy for validation set 1: 19.69%
Neural Network accuracy on test set: 19.72%
[21,   1] loss: 0.004158190
Neural Network accuracy for validation set 1: 80.31%
Neural Network accuracy on test set: 80.28%
[41,   1] loss: 0.003327143
Neural Network accuracy for validation set 1: 87.54%
Neural Network accuracy on test set: 87.22%
[61,   1] loss: 0.002975570
Neural Network accuracy for validation set 1: 88.46%
Neural Network accuracy on test set: 88.06%
[81,   1] loss: 0.002814355
Neural Network accuracy for validation set 1: 89.23%
Neural Network accuracy on test set: 90.83%
[101,   1] loss: 0.002698296
Neural Network accuracy for validation set 1: 89.85%
Neural Network accuracy on test set: 91.11%
[121,   1] loss: 0.002608443
Neural Network accuracy for validation set 1: 90.77%
Neural Network accuracy on test set: 91.11%
[141,   1] loss: 0.002518990
Neural Network accuracy for validation set 1: 91.08%
Neural Network accurac

In [157]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        
        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy for test set: {round(100.0 * correct/total, 2)}%')

Neural Network accuracy for test set: 91.39%


In [158]:
show_metrics(labels_array, predict_array)

True positive:  53.0
False positive:  13.0
True negative:  276
False negative:  18.0
True positive rate (recall):  0.7464788732394366
False positive rate:  0.04498269896193772
Precision:  0.803030303030303
True negative rate (Specificity):  0.9550173010380623
F1:  0.7737226277372263
ROC-AUC:  0.8507480871387494
MCC:  0.7213694892638097


## Performance of DNN model against published datasets

In [ ]:
df_newdb
df_cm
df_hivcopred
df_geno2pheno
df_webpssm

## For Newdb dataset

In [135]:
# Create list to append data from the df
list_data_newdb = []
list_labels_newdb = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_newdb.iterrows():
    list_data_newdb.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_newdb.append(np.array(int(row.label_numeric)))

newdb_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_newdb]) # transform to torch tensors
newdb_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_newdb])

newdb_dataset = torch.utils.data.TensorDataset(newdb_tensor_x,newdb_tensor_y) # create your test dataset
newdb_dataloader = torch.utils.data.DataLoader(newdb_dataset, batch_size=64) # create your dataloader

In [159]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in newdb_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)
print('Finished Training')

Neural Network accuracy on test set: 99.03%
True positive:  646.0
False positive:  12.0
True negative:  2323
False negative:  17.0
True positive rate (recall):  0.9743589743589743
False positive rate:  0.005139186295503212
Precision:  0.9817629179331308
True negative rate (Specificity):  0.9948608137044967
F1:  0.9780469341408025
ROC-AUC:  0.9846098940317356
MCC:  0.9718552911349981
Finished Training


## For CM dataset

In [139]:
# Create list to append data from the df
list_data_cm = []
list_labels_cm = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_cm.iterrows():
    list_data_cm.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_cm.append(np.array(int(row.label_numeric)))

cm_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_cm]) # transform to torch tensors
cm_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_cm])

cm_dataset = torch.utils.data.TensorDataset(cm_tensor_x,cm_tensor_y) # create your test dataset
cm_dataloader = torch.utils.data.DataLoader(cm_dataset, batch_size=64) # create your dataloader

In [160]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in cm_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on test set: 98.02%
True positive:  319.0
False positive:  47.0
True negative:  2307
False negative:  6.0
True positive rate (recall):  0.9815384615384616
False positive rate:  0.019966015293118096
Precision:  0.8715846994535519
True negative rate (Specificity):  0.9800339847068819
F1:  0.9232995658465991
ROC-AUC:  0.9807862231226717
MCC:  0.9141102122892164


## For hivcopred dataset

In [147]:
# Create list to append data from the df
list_data_hivcopred = []
list_labels_hivcopred = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_hivcopred.iterrows():
    list_data_hivcopred.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_hivcopred.append(np.array(int(row.label_numeric)))

hivcopred_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_hivcopred]) # transform to torch tensors
hivcopred_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_hivcopred])

hivcopred_dataset = torch.utils.data.TensorDataset(hivcopred_tensor_x,hivcopred_tensor_y) # create your test dataset
hivcopred_dataloader = torch.utils.data.DataLoader(hivcopred_dataset, batch_size=64) # create your dataloader

In [161]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in hivcopred_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on hivcopred set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on hivcopred set: 97.86%
True positive:  529.0
False positive:  12.0
True negative:  1756
False negative:  38.0
True positive rate (recall):  0.9329805996472663
False positive rate:  0.006787330316742082
Precision:  0.977818853974122
True negative rate (Specificity):  0.9932126696832579
F1:  0.9548736462093863
ROC-AUC:  0.9630966346652621
MCC:  0.9412921319180905


## For geno2pheno dataset

In [148]:
# Create list to append data from the df
list_data_geno2pheno = []
list_labels_geno2pheno = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_geno2pheno.iterrows():
    list_data_geno2pheno.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_geno2pheno.append(np.array(int(row.label_numeric)))

geno2pheno_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_geno2pheno]) # transform to torch tensors
geno2pheno_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_geno2pheno])

geno2pheno_dataset = torch.utils.data.TensorDataset(geno2pheno_tensor_x,geno2pheno_tensor_y) # create your test dataset
geno2pheno_dataloader = torch.utils.data.DataLoader(geno2pheno_dataset, batch_size=64) # create your dataloader

In [162]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in geno2pheno_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on hivcopred set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on hivcopred set: 95.96%
True positive:  191.0
False positive:  24.0
True negative:  949
False negative:  24.0
True positive rate (recall):  0.8883720930232558
False positive rate:  0.024665981500513873
Precision:  0.8883720930232558
True negative rate (Specificity):  0.9753340184994861
F1:  0.8883720930232558
ROC-AUC:  0.9318530557613709
MCC:  0.8637061115227419


## For webpssm

In [150]:
# Create list to append data from the df
list_data_webpssm = []
list_labels_webpssm = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_webpssm.iterrows():
    list_data_webpssm.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_webpssm.append(np.array(int(row.label_numeric)))

webpssm_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_webpssm]) # transform to torch tensors
webpssm_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_webpssm])

webpssm_dataset = torch.utils.data.TensorDataset(webpssm_tensor_x,webpssm_tensor_y) # create your test dataset
webpssm_dataloader = torch.utils.data.DataLoader(webpssm_dataset, batch_size=64) # create your dataloader

In [164]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in webpssm_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on webpssm set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on webpssm set: 96.29%
True positive:  69.0
False positive:  7.0
True negative:  268
False negative:  6.0
True positive rate (recall):  0.92
False positive rate:  0.025454545454545455
Precision:  0.9078947368421053
True negative rate (Specificity):  0.9745454545454545
F1:  0.9139072847682119
ROC-AUC:  0.9472727272727273
MCC:  0.8902609299817448
